In [1]:
import requests
from urllib.parse import urlparse
from io import BytesIO
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt
import os, time, uuid

In [2]:
from azure.cognitiveservices.vision.customvision.training import CustomVisionTrainingClient
from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient
from msrest.authentication import ApiKeyCredentials

In [3]:
TRAINING_ENDPOINT = "YOUR_TRAINING_ENDPOINT"
training_key = "YOUR_TRAINING_KEY"
training_resource_id = 'YOUR_TRAINING_RESOURCE_ID'

In [4]:
PREDICTION_ENDPOINT = 'YOUR_PREDICTION_ENDPOINT'
prediction_key = "YOUR_PREDICTION_KEY"
prediction_resource_id = "YOUR_PREDICTION_RESOURCE_ID"

In [5]:
training_credentials = ApiKeyCredentials(in_headers={"Training-key": training_key})
trainer = CustomVisionTrainingClient(TRAINING_ENDPOINT, training_credentials)

In [6]:
    trainer.api_version

'3.4-preview'

In [7]:
# Find the object detection domain
obj_detection_domain = next(domain for domain in trainer.get_domains() if domain.type == "ObjectDetection" and domain.name == "General")

# Create a new project
print ("Your Object Detection Training project has been created. Please move on.")
project_name = "Lighter detection"
project = trainer.create_project(project_name, domain_id=obj_detection_domain.id)

Your Object Detection Training project has been created. Please move on.


In [8]:
project.status

'Succeeded'

In [9]:
lighter_tag = trainer.create_tag(project.id, "Lighter")

In [16]:
iteration = trainer.train_project(project.id)
while (iteration.status != "Completed"):
    iteration = trainer.get_iteration(project.id, iteration.id)
    print ("Training status: " + iteration.status)
    print ("Waiting 10 seconds...")
    time.sleep(10)

Training status: Training
Waiting 10 seconds...
Training status: Training
Waiting 10 seconds...
Training status: Training
Waiting 10 seconds...
Training status: Training
Waiting 10 seconds...
Training status: Training
Waiting 10 seconds...
Training status: Training
Waiting 10 seconds...
Training status: Training
Waiting 10 seconds...
Training status: Training
Waiting 10 seconds...
Training status: Training
Waiting 10 seconds...
Training status: Training
Waiting 10 seconds...
Training status: Training
Waiting 10 seconds...
Training status: Training
Waiting 10 seconds...
Training status: Training
Waiting 10 seconds...
Training status: Training
Waiting 10 seconds...
Training status: Training
Waiting 10 seconds...
Training status: Training
Waiting 10 seconds...
Training status: Training
Waiting 10 seconds...
Training status: Training
Waiting 10 seconds...
Training status: Training
Waiting 10 seconds...
Training status: Training
Waiting 10 seconds...
Training status: Training
Waiting 10 sec

In [48]:
iteration.as_dict()

{'id': 'e36edadf-2028-4b0f-826b-cfa6a0e2bedd',
 'name': 'Iteration 2',
 'status': 'Completed',
 'created': '2022-11-01T12:59:51.473Z',
 'last_modified': '2022-11-01T13:51:27.925Z',
 'trained_at': '2022-11-01T13:51:27.920Z',
 'project_id': '2580b9ba-4c18-4aa4-bc18-cf074bdd36d3',
 'exportable': False,
 'domain_id': 'da2e3a8a-40a5-4171-82f4-58522f70fbc1',
 'training_type': 'Regular',
 'reserved_budget_in_hours': 0,
 'training_time_in_minutes': 3}

In [53]:
iteration_list = trainer.get_iterations(project.id)
train_iteration = []
for iteration_item in iteration_list:
    print(iteration_item)
    train_iteration.append(iteration_item)
    print("=====================================")

{'additional_properties': {}, 'id': 'cce6b14a-86e5-4327-b364-6df33609055f', 'name': 'Iteration 3', 'status': 'Completed', 'created': datetime.datetime(2022, 11, 1, 13, 44, 47, 983000, tzinfo=<isodate.tzinfo.Utc object at 0x0000025E98347CD0>), 'last_modified': datetime.datetime(2022, 11, 1, 14, 2, 58, 631000, tzinfo=<isodate.tzinfo.Utc object at 0x0000025E98347CD0>), 'trained_at': datetime.datetime(2022, 11, 1, 14, 2, 58, 626000, tzinfo=<isodate.tzinfo.Utc object at 0x0000025E98347CD0>), 'project_id': '2580b9ba-4c18-4aa4-bc18-cf074bdd36d3', 'exportable': False, 'exportable_to': None, 'domain_id': 'da2e3a8a-40a5-4171-82f4-58522f70fbc1', 'classification_type': None, 'training_type': 'Regular', 'reserved_budget_in_hours': 0, 'training_time_in_minutes': 3, 'publish_name': None, 'original_publish_resource_id': None, 'custom_base_model_info': None, 'training_error_details': None}
{'additional_properties': {}, 'id': 'e36edadf-2028-4b0f-826b-cfa6a0e2bedd', 'name': 'Iteration 2', 'status': 'Comp

In [57]:
train_iteration[0].id, iteration_list[0].id

('cce6b14a-86e5-4327-b364-6df33609055f',
 'cce6b14a-86e5-4327-b364-6df33609055f')

In [58]:
model_perf = trainer.get_iteration_performance(project.id, iteration_list[0].id)

In [59]:
model_perf.as_dict()

{'per_tag_performance': [{'id': '5da234ae-2090-45eb-9d69-22a7a79237e3',
   'name': 'Lighter',
   'precision': 1.0,
   'precision_std_deviation': 0.0,
   'recall': 0.3846154,
   'recall_std_deviation': 0.0,
   'average_precision': 0.7761013}],
 'precision': 1.0,
 'precision_std_deviation': 0.0,
 'recall': 0.3846154,
 'recall_std_deviation': 0.0,
 'average_precision': 0.7761013}

### Publish the model

In [61]:
# iteration_id = model_perf.as_dict()['per_tag_performance'][0]['id']
iteration_id = iteration_list[0].id
iteration_id

'cce6b14a-86e5-4327-b364-6df33609055f'

In [62]:
publish_iteration_name = "lighter-object-detection-custom"

In [63]:
trainer.publish_iteration(project.id, iteration_id, publish_iteration_name, prediction_resource_id)
print ("Done!")

Done!


### Perform Prediction

In [64]:
prediction_credentials = ApiKeyCredentials(in_headers={"Prediction-key": prediction_key})
predictor = CustomVisionPredictionClient(PREDICTION_ENDPOINT, prediction_credentials)

In [69]:
local_image_path = 'lighter_test_images'

In [70]:
def perform_prediction(image_file_name):
    with open(os.path.join (local_image_path,  image_file_name), "rb") as image_contents:
        results = predictor.detect_image(project.id, publish_iteration_name, image_contents.read())
        # Display the results.
        for prediction in results.predictions:
            print("\t" + prediction.tag_name +
                  ": {0:.2f}%".format(prediction.probability * 100))

In [71]:
file_name = "lighter_test_set_1of5.jpg"

In [72]:
perform_prediction(file_name)

	Lighter: 65.39%
	Lighter: 11.85%
	Lighter: 9.99%
	Lighter: 5.93%
	Lighter: 5.77%
	Lighter: 3.31%
	Lighter: 2.74%
	Lighter: 2.39%
	Lighter: 2.01%
	Lighter: 1.99%
	Lighter: 1.97%
	Lighter: 1.60%
	Lighter: 1.52%
	Lighter: 1.46%
	Lighter: 1.37%
	Lighter: 1.23%
	Lighter: 1.15%
	Lighter: 1.11%
	Lighter: 1.09%
	Lighter: 1.06%
	Lighter: 0.99%
	Lighter: 0.89%
	Lighter: 0.89%
	Lighter: 0.86%
	Lighter: 0.83%
	Lighter: 0.82%
	Lighter: 0.82%
	Lighter: 0.82%
	Lighter: 0.79%
	Lighter: 0.78%
	Lighter: 0.77%
	Lighter: 0.72%
	Lighter: 0.72%
	Lighter: 0.70%
	Lighter: 0.70%
	Lighter: 0.69%
	Lighter: 0.68%
	Lighter: 0.64%
	Lighter: 0.63%
	Lighter: 0.60%
	Lighter: 0.60%
	Lighter: 0.59%
	Lighter: 0.58%
	Lighter: 0.58%
	Lighter: 0.55%
	Lighter: 0.54%
	Lighter: 0.54%
	Lighter: 0.54%
	Lighter: 0.53%
	Lighter: 0.52%


In [74]:
import glob, os, sys, time, uuid

In [79]:
human_face_images = os.listdir('lighter_test_images')
human_face_images

['lighter_test_set_1of5.jpg',
 'lighter_test_set_2of5.jpg',
 'lighter_test_set_3of5.jpg',
 'lighter_test_set_4of5.jpg',
 'lighter_test_set_5of5.jpg']

In [92]:
def perform_prediction_on_multiple_images(image_folder):
    image_list = os.listdir(image_folder)
    for image_file_name in image_list:
        with open(os.path.join (image_folder,  image_file_name), "rb") as image_contents:
            results = predictor.detect_image(project.id, publish_iteration_name, image_contents.read())
            # Display the results.
            for prediction in results.predictions:
                print("\t" + prediction.tag_name + ": {0:.2f}% bbox.left = {1:.2f}, bbox.top = {2:.2f}, bbox.width = {3:.2f}, bbox.height = {4:.2f}".format(prediction.probability * 100, prediction.bounding_box.left, prediction.bounding_box.top, prediction.bounding_box.width, prediction.bounding_box.height))
            print(f"=================== Image name => {image_file_name} ======================================================")

In [93]:
perform_prediction_on_multiple_images(local_image_path)

	Lighter: 65.39% bbox.left = 0.33, bbox.top = 0.05, bbox.width = 0.67, bbox.height = 0.91
	Lighter: 11.85% bbox.left = 0.10, bbox.top = 0.11, bbox.width = 0.33, bbox.height = 0.76
	Lighter: 9.99% bbox.left = 0.35, bbox.top = 0.62, bbox.width = 0.45, bbox.height = 0.36
	Lighter: 5.93% bbox.left = 0.15, bbox.top = 0.77, bbox.width = 0.05, bbox.height = 0.09
	Lighter: 5.77% bbox.left = 0.10, bbox.top = 0.77, bbox.width = 0.05, bbox.height = 0.09
	Lighter: 3.31% bbox.left = 0.15, bbox.top = 0.69, bbox.width = 0.05, bbox.height = 0.09
	Lighter: 2.74% bbox.left = 0.10, bbox.top = 0.82, bbox.width = 0.05, bbox.height = 0.10
	Lighter: 2.39% bbox.left = 0.26, bbox.top = 0.44, bbox.width = 0.05, bbox.height = 0.10
	Lighter: 2.01% bbox.left = 0.15, bbox.top = 0.83, bbox.width = 0.05, bbox.height = 0.09
	Lighter: 1.99% bbox.left = 0.10, bbox.top = 0.69, bbox.width = 0.05, bbox.height = 0.09
	Lighter: 1.97% bbox.left = 0.20, bbox.top = 0.77, bbox.width = 0.05, bbox.height = 0.09
	Lighter: 1.60% bbo